In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [2]:
%%javascript
IPython.notebook.clear_all_output();

<IPython.core.display.Javascript object>

In [3]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

## Importing Necessary Libraries

In [4]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split

print("tensorflow version : ", tf.__version__)
print("tensorflow_hub version : ", hub.__version__)

tensorflow version :  1.14.0
tensorflow_hub version :  0.7.0


In [5]:
#Importing BERT modules
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [6]:
import tensorflow.contrib.eager as tfe


## Setting The Output Directory

While fine-tuning the model, we will save the training checkpoints and the model in an output directory so that we can use the trained model for our predictions later.

In [7]:
# Set the output directory for saving model file
OUTPUT_DIR = 'model_pooled_concat_sequence_norelabel_2_layer_ms0'

#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: model_pooled_concat_sequence_norelabel_2_layer_ms0 *****


## Loading The Data

In [8]:
data=pd.read_csv('noneed_relabel.csv',index_col=0)
data

,text,search_term,label0,label1,label2
4,Tesla and Microsoft topped Wall Street earning...,MSFT,1,1.0,1.0
11,Microsoft ex-employee says contractors in Chin...,Microsoft,1,-1.0,-1.0
13,"Stocks making the biggest moves midday: Tesla,...",Microsoft,1,1.0,1.0
17,Microsoft says it aims to remove more carbon f...,Microsoft,1,0.5,1.0
19,"Salesforce, which once tried to buy LinkedIn, ...",Microsoft,1,0.0,0.0
22,Microsoft health's Peter Lee says medical indu...,Microsoft,0,0.0,0.0
23,Apple and Microsoft representatives are set to...,Microsoft,1,1.0,1.0
32,Microsoft CEO's recommends these 2 morning rit...,Microsoft,1,0.0,0.0
33,"In this #CNBCClassic clip from 2000, Bill Gate...",Microsoft,1,0.0,0.0
39,Slack shares dip as Microsoft prepares global ...,Microsoft,1,1.0,1.0


In [10]:
data['label2'].value_counts()

 0.0    982
 1.0    677
-1.0    565
-0.5    279
 0.5    260
-1.5     25
-2.0     24
 1.5     18
Name: label2, dtype: int64

In [11]:
for i in range(len(data)):
    if data.iloc[i,4]>0:
        data.iloc[i,4]=1
    elif data.iloc[i,4]==0:
        data.iloc[i,4]=0
    else:
        data.iloc[i,4]=-1
data

,text,search_term,label0,label1,label2
4,Tesla and Microsoft topped Wall Street earning...,MSFT,1,1.0,1.0
11,Microsoft ex-employee says contractors in Chin...,Microsoft,1,-1.0,-1.0
13,"Stocks making the biggest moves midday: Tesla,...",Microsoft,1,1.0,1.0
17,Microsoft says it aims to remove more carbon f...,Microsoft,1,0.5,1.0
19,"Salesforce, which once tried to buy LinkedIn, ...",Microsoft,1,0.0,0.0
22,Microsoft health's Peter Lee says medical indu...,Microsoft,0,0.0,0.0
23,Apple and Microsoft representatives are set to...,Microsoft,1,1.0,1.0
32,Microsoft CEO's recommends these 2 morning rit...,Microsoft,1,0.0,0.0
33,"In this #CNBCClassic clip from 2000, Bill Gate...",Microsoft,1,0.0,0.0
39,Slack shares dip as Microsoft prepares global ...,Microsoft,1,1.0,1.0


In [12]:
data['label2'].value_counts()

 0.0    982
 1.0    955
-1.0    893
Name: label2, dtype: int64

In [13]:
data_flag=data['label2']
data_feature=data
data_feature.drop('label2', axis = 1, inplace = True)

## split dataset to train test and val

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test_val, y_train, y_test_val=train_test_split(data_feature,data_flag,test_size=0.3,random_state=42)
print(X_train.shape)
print(X_test_val.shape)
print(y_train.shape)
print(y_test_val.shape)

(1981, 4)
(849, 4)
(1981,)
(849,)


In [15]:
X_test, X_val, y_test, y_val=train_test_split(X_test_val,y_test_val,test_size=0.33,random_state=42)
print(X_test.shape)
print(X_val.shape)
print(y_test.shape)
print(y_val.shape)

(568, 4)
(281, 4)
(568,)
(281,)


In [16]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(1981, 4)
(1981,)
(568, 4)
(568,)
(281, 4)
(281,)


In [17]:
y_train.value_counts()

 0.0    684
 1.0    680
-1.0    617
Name: label2, dtype: int64

In [18]:
search_term=list(X_train['search_term'])
len(search_term)

1981

In [19]:
X_train['label0']=X_train['search_term']
X_train['search_term']=y_train
X_train

/home/lateef11/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/lateef11/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,search_term,label0,label1
7015,For subscribers: Disney shares drop after two ...,-1.0,Disney,-1.0
5769,Five large financial firms from the US and the...,1.0,Amazon,1.0
4812,Boeing plans full drawdown of $13.8 billion lo...,-1.0,Boeing,-1.0
8228,China has tripled its presence on Facebook and...,-1.0,Facebook,-1.0
4405,New for subscribers: Wells Fargo upgrades Appl...,1.0,Apple,1.0
3861,Three stories to read this morning... Newborn ...,0.0,Microsoft,0.0
3391,How sneaker fans are cashing in on the $2bn re...,0.0,Nike,0.0
2840,"5 things to know about GM's new Yukon, Yukon X...",0.0,GM,0.0
9556,The failures and successes that have kept Nint...,0.0,Nintendo,0.0
9416,'Fortnite' Spy Games guide: how to gather Inte...,0.0,SPY,0.0


In [20]:
X_train['search_term'].value_counts()

 0.0    684
 1.0    680
-1.0    617
Name: search_term, dtype: int64

In [21]:
X_test['label0']=X_test['search_term']
X_test['search_term']=y_test
X_val['label0']=X_val['search_term']
X_val['search_term']=y_val

/home/lateef11/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/lateef11/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/lateef11/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [22]:
train=X_train
val=X_val
test=X_test

In [23]:
train.columns=['clean tweet','target','search_term','label1']
val.columns=['clean tweet','target','search_term','label1']
test.columns=['clean tweet','target','search_term','label1']

In [24]:
train

,clean tweet,target,search_term,label1
7015,For subscribers: Disney shares drop after two ...,-1.0,Disney,-1.0
5769,Five large financial firms from the US and the...,1.0,Amazon,1.0
4812,Boeing plans full drawdown of $13.8 billion lo...,-1.0,Boeing,-1.0
8228,China has tripled its presence on Facebook and...,-1.0,Facebook,-1.0
4405,New for subscribers: Wells Fargo upgrades Appl...,1.0,Apple,1.0
3861,Three stories to read this morning... Newborn ...,0.0,Microsoft,0.0
3391,How sneaker fans are cashing in on the $2bn re...,0.0,Nike,0.0
2840,"5 things to know about GM's new Yukon, Yukon X...",0.0,GM,0.0
9556,The failures and successes that have kept Nint...,0.0,Nintendo,0.0
9416,'Fortnite' Spy Games guide: how to gather Inte...,0.0,SPY,0.0


In [25]:
train['target'].value_counts()

 0.0    684
 1.0    680
-1.0    617
Name: target, dtype: int64

In [26]:
max_value=0
for i in train['target'].value_counts():
    if i>max_value:
        max_value=i
max_value

684

In [27]:
train_0_down=train[train['target']==0].index
train_0_down

Int64Index([3861, 3391, 2840, 9556, 9416, 5706, 2090, 3123, 8612,  898,
            ...
            2984, 9713, 9169, 1460, 2269, 3537, 8689, 4650,  424, 3849],
           dtype='int64', length=684)

In [28]:
train_1_down=train[train['target']==1].index
train_1_down

Int64Index([5769, 4405, 5538,  670, 1804, 3016,  237, 6816, 3046, 3612,
            ...
            3286, 8208, 2890, 3594, 5246, 7497, 3668, 5052, 3318, 3438],
           dtype='int64', length=680)

In [29]:
train_m1_down=train[train['target']==-1].index
train_m1_down

Int64Index([7015, 4812, 8228, 6562, 3634, 4689, 5060,  997,  566, 1009,
            ...
            8127,  111, 1422, 8461, 2316, 4436,  961, 1441, 7640, 2574],
           dtype='int64', length=617)

In [30]:
import random
slice1 = random.sample(list(train_0_down), max_value-len(train_0_down))
slice2 = random.sample(list(train_1_down), max_value-len(train_1_down))
slice3 = random.sample(list(train_m1_down), max_value-len(train_m1_down))
slice4=slice1+slice2+slice3

In [31]:
for i in slice4:
    df=train[train.index==i]
    df.index=df.index+15000
    train=pd.concat([train,df])

In [32]:
train['target'].value_counts()

 0.0    684
 1.0    684
-1.0    684
Name: target, dtype: int64

In [33]:
print("Training Set Shape :", train.shape)
print("Validation Set Shape :", val.shape)
print("Test Set Shape :", test.shape)

Training Set Shape : (2052, 4)
Validation Set Shape : (281, 4)
Test Set Shape : (568, 4)


In [34]:
#unique classes
train['target'].unique()

array([-1.,  1.,  0.])

In [35]:
#Distribution of classes
train['target'].value_counts().plot(kind = 'bar')

In [36]:
DATA_COLUMN = 'clean tweet'
LABEL_COLUMN = 'target'
# The list containing all the classes (train['SECTION'].unique())
label_list = [-1,0, 1]

In [37]:
redeal=['ZM','XOM','Walmart','Virgin Galactic','United Airlines','USO','Spirit Airlines','Southwest Airlines',
 'SIRI','Royal Caribbean','ROKU','Pfizer','Occidental Petroleum','Norwegian Cruise Line','New Residential','Moderna','Marathon Oil',
 'Luckin Coffee','LYFT','LUV','LK','JetBlue','JPM','Halliburton','Groupon','GoPro','Gilead Sciences','Fitbit','Exxon Mobil',
 'Energy Transfer','Delta Air Lines','Coca-Cola','Canopy Growth','CCL','Beyond Meat','Berkshire Hathaway','Bank of America','BYND',
 'Aurora Cannabis','American Airlines','Alibaba','AMD','DIS']

In [38]:
for i in range(len(train)):
    if train.iloc[i,2] in redeal:
        cleaning=train.iloc[i,0].lower()
        cleaning2=train.iloc[i,2].lower()
        tweets=cleaning.replace(cleaning2,'presentation')
        train.iloc[i,0]=tweets
        train.iloc[i,2]='presentation'
train

,clean tweet,target,search_term,label1
7015,For subscribers: Disney shares drop after two ...,-1.0,Disney,-1.0
5769,Five large financial firms from the US and the...,1.0,Amazon,1.0
4812,Boeing plans full drawdown of $13.8 billion lo...,-1.0,Boeing,-1.0
8228,China has tripled its presence on Facebook and...,-1.0,Facebook,-1.0
4405,New for subscribers: Wells Fargo upgrades Appl...,1.0,Apple,1.0
3861,Three stories to read this morning... Newborn ...,0.0,Microsoft,0.0
3391,How sneaker fans are cashing in on the $2bn re...,0.0,Nike,0.0
2840,"5 things to know about GM's new Yukon, Yukon X...",0.0,GM,0.0
9556,The failures and successes that have kept Nint...,0.0,Nintendo,0.0
9416,'Fortnite' Spy Games guide: how to gather Inte...,0.0,SPY,0.0


In [39]:
for i in range(len(val)):
    if val.iloc[i,2] in redeal:
        cleaning=val.iloc[i,0].lower()
        cleaning2=val.iloc[i,2].lower()
        tweets=cleaning.replace(cleaning2,'presentation')
        val.iloc[i,0]=tweets
        val.iloc[i,2]='presentation'
val

/home/lateef11/.conda/envs/deep/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,clean tweet,target,search_term,label1
8641,5 Amazon workers told CNBC they’ve received no...,-1.0,Amazon,-1.0
9267,Nearly half of American households already sub...,1.0,Netflix,1.0
3336,Two men jailed for helping Iran spy on disside...,0.0,SPY,0.0
9162,All the stars of Netflix's 'Outer Banks' and w...,0.0,Netflix,0.0
9656,NHL condemns 'vile' racist outburst towards bl...,0.0,Zoom,0.0
9961,"What did they know, exactly? US intel warned o...",0.0,Intel,0.0
1008,'My stomach turned' — Boeing CEO outraged by i...,-1.0,Boeing,-1.0
6757,"presentation plans to hire 150,000 temporary w...",1.0,presentation,1.0
9152,If you want to make a movie Netflix says there...,0.0,Netflix,0.0
7560,Coronavirus: BA union pledges to flight for ev...,1.0,BA,1.0


In [40]:
for i in range(len(test)):
    if test.iloc[i,2] in redeal:
        cleaning=test.iloc[i,0].lower()
        cleaning2=test.iloc[i,2].lower()
        tweets=cleaning.replace(cleaning2,'presentation')
        test.iloc[i,0]=tweets
        test.iloc[i,2]='presentation'
test

/home/lateef11/.conda/envs/deep/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,clean tweet,target,search_term,label1
1371,What does the new boss of Boeing need to do to...,0.0,Boeing,0.0
3024,Stocks making the biggest moves after hours: I...,1.0,Netflix,1.0
7678,"Boeing plans to cut 787 Dreamliner output, job...",-1.0,Boeing,-1.0
9766,"A surge in Zoom-bombing, or uninvited guests d...",-1.0,Zoom,-1.0
433,presentation sued by california teachers over ...,-1.0,presentation,-2.0
9945,IBM's new CEO Arvind Krishna said he will focu...,0.0,IBM,0.0
5525,"""vanish, o night! at dawn, i will win!"" china'...",1.0,presentation,1.0
9962,Stocks making the biggest moves midday: JCPenn...,0.0,Intel,0.0
3657,IBM soars on strong earnings report.,1.0,IBM,1.0
7863,"Tesla will slash employee pay, and will furlou...",-1.0,Tesla,-1.0


## Data Preprocessing

BERT model accept only a specific type of input and the datasets are usually structured to have the following four features:

* guid : A unique id that represents an observation.
* text_a : The text we need to classify into given categories
* text_b: It is used when we're training a model to understand the relationship between sentences and it does not apply for classification problems.
* label: It consists of the labels or classes or categories that a given text belongs to.
 
In our dataset we have text_a and label. The following code block will create objects for each of the above mentioned features for all the records in our dataset using the InputExample class provided in the BERT library.


In [41]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [42]:
test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [43]:
print("### Row 0 - guid of training set : ", train_InputExamples.iloc[0].guid)
print("\n### Row 0 - text_a of training set : \n", train_InputExamples.iloc[0].text_a)
print("\n### Row 0 - text_b of training set : ", train_InputExamples.iloc[0].text_b)
print("\n### Row 0 - label of training set : ", train_InputExamples.iloc[0].label)

### Row 0 - guid of training set :  None

### Row 0 - text_a of training set : 
 For subscribers: Disney shares drop after two firms downgrade on concern about lost parks revenue 

### Row 0 - text_b of training set :  None

### Row 0 - label of training set :  -1.0


## Tokenizing Texts

We will now get down to business with the pretrained BERT.  In this example we will use the ```bert_uncased_L-12_H-768_A-12/1``` model. To check all available versions click [here](https://tfhub.dev/s?network-architecture=transformer&publisher=google).

We will be using the vocab.txt file in the model to map the words in the dataset to indexes. Also the loaded BERT model is trained on uncased/lowercase data and hence the data we feed to train the model should also be of lowercase.

---

The following code block loads the pre-trained BERT model and initializers a tokenizer object for tokenizing the texts.


In [44]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
        
    # !!! this is time-consuming as the model is giant !!!
    bert_module = hub.Module(BERT_MODEL_HUB) # second time is fast as the module is cached
    
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [45]:
#Here is what the tokenised sample of the first training set observation looks like
print(tokenizer.tokenize(train_InputExamples.iloc[0].text_a))

['for', 'subscribers', ':', 'disney', 'shares', 'drop', 'after', 'two', 'firms', 'down', '##grade', 'on', 'concern', 'about', 'lost', 'parks', 'revenue']


In [46]:
train

,clean tweet,target,search_term,label1
7015,For subscribers: Disney shares drop after two ...,-1.0,Disney,-1.0
5769,Five large financial firms from the US and the...,1.0,Amazon,1.0
4812,Boeing plans full drawdown of $13.8 billion lo...,-1.0,Boeing,-1.0
8228,China has tripled its presence on Facebook and...,-1.0,Facebook,-1.0
4405,New for subscribers: Wells Fargo upgrades Appl...,1.0,Apple,1.0
3861,Three stories to read this morning... Newborn ...,0.0,Microsoft,0.0
3391,How sneaker fans are cashing in on the $2bn re...,0.0,Nike,0.0
2840,"5 things to know about GM's new Yukon, Yukon X...",0.0,GM,0.0
9556,The failures and successes that have kept Nint...,0.0,Nintendo,0.0
9416,'Fortnite' Spy Games guide: how to gather Inte...,0.0,SPY,0.0


In [47]:
train_part=train
train_part

,clean tweet,target,search_term,label1
7015,For subscribers: Disney shares drop after two ...,-1.0,Disney,-1.0
5769,Five large financial firms from the US and the...,1.0,Amazon,1.0
4812,Boeing plans full drawdown of $13.8 billion lo...,-1.0,Boeing,-1.0
8228,China has tripled its presence on Facebook and...,-1.0,Facebook,-1.0
4405,New for subscribers: Wells Fargo upgrades Appl...,1.0,Apple,1.0
3861,Three stories to read this morning... Newborn ...,0.0,Microsoft,0.0
3391,How sneaker fans are cashing in on the $2bn re...,0.0,Nike,0.0
2840,"5 things to know about GM's new Yukon, Yukon X...",0.0,GM,0.0
9556,The failures and successes that have kept Nint...,0.0,Nintendo,0.0
9416,'Fortnite' Spy Games guide: how to gather Inte...,0.0,SPY,0.0


In [48]:
val_part=val
val_part

,clean tweet,target,search_term,label1
8641,5 Amazon workers told CNBC they’ve received no...,-1.0,Amazon,-1.0
9267,Nearly half of American households already sub...,1.0,Netflix,1.0
3336,Two men jailed for helping Iran spy on disside...,0.0,SPY,0.0
9162,All the stars of Netflix's 'Outer Banks' and w...,0.0,Netflix,0.0
9656,NHL condemns 'vile' racist outburst towards bl...,0.0,Zoom,0.0
9961,"What did they know, exactly? US intel warned o...",0.0,Intel,0.0
1008,'My stomach turned' — Boeing CEO outraged by i...,-1.0,Boeing,-1.0
6757,"presentation plans to hire 150,000 temporary w...",1.0,presentation,1.0
9152,If you want to make a movie Netflix says there...,0.0,Netflix,0.0
7560,Coronavirus: BA union pledges to flight for ev...,1.0,BA,1.0


In [49]:
test_part=test
test_part

,clean tweet,target,search_term,label1
1371,What does the new boss of Boeing need to do to...,0.0,Boeing,0.0
3024,Stocks making the biggest moves after hours: I...,1.0,Netflix,1.0
7678,"Boeing plans to cut 787 Dreamliner output, job...",-1.0,Boeing,-1.0
9766,"A surge in Zoom-bombing, or uninvited guests d...",-1.0,Zoom,-1.0
433,presentation sued by california teachers over ...,-1.0,presentation,-2.0
9945,IBM's new CEO Arvind Krishna said he will focu...,0.0,IBM,0.0
5525,"""vanish, o night! at dawn, i will win!"" china'...",1.0,presentation,1.0
9962,Stocks making the biggest moves midday: JCPenn...,0.0,Intel,0.0
3657,IBM soars on strong earnings report.,1.0,IBM,1.0
7863,"Tesla will slash employee pay, and will furlou...",-1.0,Tesla,-1.0


In [50]:
search_dict={'msft':'microsoft','ford':'ford','ge':'ge','dis':'disney','aal':'american airlines','dal':'delta air lines',\
               'gpro':'gopro','ccl':'carnival','aapl':'apple','fitbit':'fitbit','plug':'plug power','bac':'bank of america',\
               'snap':'snap','nclh':'norwegian cruise line','ba':'boeing','ino':'inovio','ual':'united airlines','cgc':\
                'canopy growth','uber':'uber','rcl':'royal caribbean','cron':'cronos group','amd':'amd','twtr':'twtr','tsla':\
               'tesla','grpn':'groupon','fb':'facebook','acb':'aurora cannabis','sbux':'starbucks','mro':'marathon oil','delta':'delta',\
                'znga':'zynga','baba':'alibaba','uso':'united states oil fund','ko':'coca-cola','apha':'aphria',\
                'xom':'exxon mobil','mfa':'mfa','jblu':'jetblue','amzn':'amazon','luv':'southwest airlines','nio':'nio','mrna':'moderna',\
                'gm':'gm','spirit airlines':'spirit Airlines','mgm':'mgm','gild':'gilead sciences','nflx':'netflix','nrz':'new residential','spce':\
                'virgin galactic','lk':'luckin coffee','vslr':'vivint solar','uco':'uco','voo':'vanguard','penn':'penn','amc':'amc','tlry':\
                'tilray','hal':'halliburton','nok':'nokia','lyft':'lyft','nvda':'nvidia','cprx':'catalyst pharmaceuticals','spy':\
                'spdr s&p 500 etf','nke':'nike','sq':'sq','visa':'visa','siri':'siri','brk.b':'berkshire hathaway','cpe':'callon petroleum',\
                'slack':'slack','gush':'gush','bynd':'beyond meat','pfe':'pfizer','oxy':'occidental petroleum','kos':'kos',\
                'energy transfer':'energy transfer','crbp':'corbus pharmaceuticals','sphd':'sphd','ntdoy':'nintendo','zm':'zoom','jpm':'jpm',\
                'vktx':'viking therapeutics','ibm':'ibm','nymt':'nymt','intc':'intel','wmt':'walmart','sne':'sony','roku':'roku','presentation':'presentation'}
search_dict

{'msft': 'microsoft',
 'ford': 'ford',
 'ge': 'ge',
 'dis': 'disney',
 'aal': 'american airlines',
 'dal': 'delta air lines',
 'gpro': 'gopro',
 'ccl': 'carnival',
 'aapl': 'apple',
 'fitbit': 'fitbit',
 'plug': 'plug power',
 'bac': 'bank of america',
 'snap': 'snap',
 'nclh': 'norwegian cruise line',
 'ba': 'boeing',
 'ino': 'inovio',
 'ual': 'united airlines',
 'cgc': 'canopy growth',
 'uber': 'uber',
 'rcl': 'royal caribbean',
 'cron': 'cronos group',
 'amd': 'amd',
 'twtr': 'twtr',
 'tsla': 'tesla',
 'grpn': 'groupon',
 'fb': 'facebook',
 'acb': 'aurora cannabis',
 'sbux': 'starbucks',
 'mro': 'marathon oil',
 'delta': 'delta',
 'znga': 'zynga',
 'baba': 'alibaba',
 'uso': 'united states oil fund',
 'ko': 'coca-cola',
 'apha': 'aphria',
 'xom': 'exxon mobil',
 'mfa': 'mfa',
 'jblu': 'jetblue',
 'amzn': 'amazon',
 'luv': 'southwest airlines',
 'nio': 'nio',
 'mrna': 'moderna',
 'gm': 'gm',
 'spirit airlines': 'spirit Airlines',
 'mgm': 'mgm',
 'gild': 'gilead sciences',
 'nflx': 'n

In [51]:
len(train_InputExamples)

2052

In [52]:
binar=[]
nodrop_index=[]
errr=0
for i in range (len(train_InputExamples)):
    c=[]
    for j in tokenizer.tokenize(train_InputExamples.iloc[i].text_a):
        #print(j)
        search_1=train_part['search_term'].iloc[i]
        if search_1.lower() in search_dict.keys():
            search_2=search_dict[search_1.lower()]
        else:
            search_2=list(search_dict.keys())[list(search_dict.values()).index(search_1.lower())]
        if j == search_1.lower() or j == search_2:
            c.append(1)
        else:
            c.append(0)
    if sum(c)!=0:
        nodrop_index.append(i)
        binar.append(c)

In [53]:
train_InputExamples=train_InputExamples.iloc[nodrop_index]

In [54]:
len(train_InputExamples)

1997

In [55]:
binar2=[]
for i in range (len(val_InputExamples)):
    c=[]
    for j in tokenizer.tokenize(val_InputExamples.iloc[i].text_a):
        search_1=val_part['search_term'].iloc[i]    
        if search_1.lower() in search_dict.keys():
            search_2=search_dict[search_1.lower()]
        else:
            search_2=list(search_dict.keys())[list(search_dict.values()).index(search_1.lower())]
        if j==search_1.lower() or j==search_2:
            c.append(1)
        else:
            c.append(0)
    binar2.append(c)

In [56]:
binar3=[]
for i in range (len(test_InputExamples)):
    c=[]
    for j in tokenizer.tokenize(test_InputExamples.iloc[i].text_a):
        search_1=test_part['search_term'].iloc[i]    
        if search_1.lower() in search_dict.keys():
            search_2=search_dict[search_1.lower()]
        else:
            search_2=list(search_dict.keys())[list(search_dict.values()).index(search_1.lower())]
        if j==search_1.lower() or j==search_2:
            c.append(1)
        else:
            c.append(0)
    binar3.append(c)

In [57]:
import numpy as np
for i in binar:
     while len(i)<128:
        i.append(0)
binar=np.array(binar)

In [58]:
for i in binar2:
     while len(i)<128:
        i.append(0)
binar2=np.array(binar2)

In [59]:
for i in binar3:
     while len(i)<128:
        i.append(0)
binar3=np.array(binar3)

We will now format out text in to input features which the BERT model expects. We will also set a sequence length which will be the length of the input features.

# Model Creation

In [60]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 1997


INFO:tensorflow:Writing example 0 of 1997


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0.0 (id = 1)


INFO:tensorflow:label: 0.0 (id = 1)


In [61]:
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 568


INFO:tensorflow:Writing example 0 of 568


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: -1.0 (id = 0)


INFO:tensorflow:label: -1.0 (id = 0)


In [62]:
for i in range(len(train_features)):
    train_features[i].binary_seq=list(binar[i])

In [63]:
for i in range(len(val_features)):
    val_features[i].binary_seq=list(binar2[i])

In [64]:
for i in range(len(test_features)):
    test_features[i].binary_seq=list(binar3[i])

In [65]:
#Example on first observation in the training set
print("### Text : \n", train_InputExamples.iloc[0].text_a)
print("-"*30)
print("### Input IDs : ", train_features[0].input_ids)
print("-"*30)
print("### Input Masks : ", train_features[0].input_mask)
print("-"*30)
print("### Segment IDs : ", train_features[0].segment_ids)
print("-"*30)
print("### binary_seq : ", train_features[0].binary_seq)

### Text : 
 For subscribers: Disney shares drop after two firms downgrade on concern about lost parks revenue 
------------------------------
### Input IDs :  [101, 2005, 17073, 1024, 6373, 6661, 4530, 2044, 2048, 9786, 2091, 24170, 2006, 5142, 2055, 2439, 6328, 6599, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
### Input Masks :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Creating a Multi-Class Classifier Model

In [67]:
#tfe.enable_eager_execution()
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels,binary_seq):
  #tfe.enable_eager_execution()
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens", # bert_inputs records tokens
      as_dict=True) # bert_inputs is a dict

  # !!! Use "pooled_output" for classification tasks on an entire sentence. ---> adopted for classification
  # !!! Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]
  word_layer = bert_outputs["sequence_output"]

  binary_seq=tf.expand_dims(binary_seq, 2)
  word_layer = tf.multiply(word_layer, binary_seq)
  word_layer=tf.reduce_sum(word_layer,axis=1)
  #output_layer=tf.concat([output_layer,word_layer],axis=1)

  hidden_size = output_layer.shape[-1].value
  middle_size=[32,64,128,256,512]
  # Create our own layer to tune for politeness data.

  with tf.variable_scope("loss"):
    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
    word_layer = tf.nn.dropout(word_layer, keep_prob=0.9)
    output_layer = tf.layers.dense(output_layer, middle_size[0], activation=tf.nn.relu)
    word_layer = tf.layers.dense(word_layer, middle_size[0], activation=tf.nn.relu)
    logits=tf.concat([output_layer,word_layer],axis=1)
    logits_out=logits
    logits = tf.layers.dense(logits, num_labels)
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32) 
    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs,logits_out)
    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [68]:
# A function that adapts our model to work for training, evaluation, and prediction.

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):
    """The `model_fn` for TPUEstimator."""
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    binary_seq = features["binary_seq"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels,binary_seq)
      
      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        precision=tf.metrics.precision(label_ids, predicted_labels)
        recall=tf.metrics.recall(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "precision":precision,
            "recall":recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
            }
      eval_metrics = metric_fn(label_ids, predicted_labels)
      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:

          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)

    else:
      
      (predicted_labels, log_probs,logits_out) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels,binary_seq)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels,
          'logits_out': logits_out
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [69]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 4.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [70]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'model_pooled_concat_sequence_norelabel_2_layer_ms0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x2aabfed21518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'model_pooled_concat_sequence_norelabel_2_layer_ms0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x2aabfed21518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [71]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []
  all_binary_seq = []
  for feature in features:
    
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)
    all_binary_seq.append(feature.binary_seq)
  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
        "binary_seq":
            tf.constant(
                all_binary_seq, 
                shape=[num_examples, seq_length],
                dtype=tf.float32)
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

we will now create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [72]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

# Create an input function for validating. drop_remainder = True for using TPUs.

In [73]:
val_input_fn = input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [74]:
test_input_fn = input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

## Training & Evaluating

In [75]:
# Training the model
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)#
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 249 into model_pooled_concat_sequence_norelabel_2_layer_ms0/model.ckpt.


INFO:tensorflow:Saving checkpoints for 249 into model_pooled_concat_sequence_norelabel_2_layer_ms0/model.ckpt.


INFO:tensorflow:Loss for final step: 0.052981872.


INFO:tensorflow:Loss for final step: 0.052981872.


Training took time  0:02:51.070452


In [76]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=val_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/home/lateef11/.conda/envs/deep/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 249: eval_accuracy = 0.7935943, false_negatives = 22.0, false_positives = 17.0, global_step = 249, loss = 0.8987852, precision = 0.91099477, recall = 0.8877551, true_negatives = 68.0, true_positives = 174.0


INFO:tensorflow:Saving dict for global step 249: eval_accuracy = 0.7935943, false_negatives = 22.0, false_positives = 17.0, global_step = 249, loss = 0.8987852, precision = 0.91099477, recall = 0.8877551, true_negatives = 68.0, true_positives = 174.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 249: model_pooled_concat_sequence_norelabel_2_layer_ms0/model.ckpt-249


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 249: model_pooled_concat_sequence_norelabel_2_layer_ms0/model.ckpt-249


{'eval_accuracy': 0.7935943,
 'false_negatives': 22.0,
 'false_positives': 17.0,
 'loss': 0.8987852,
 'precision': 0.91099477,
 'recall': 0.8877551,
 'true_negatives': 68.0,
 'true_positives': 174.0,
 'global_step': 249}

## Predicting For Test Set

In [78]:
# A method to get predictions
def getPrediction2(in_sentences,test_input_fn):
  # Transforming the test data into BERT accepted form
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 1) for x in in_sentences] 
  
  # Creating input features for Test data
  predictions=estimator.predict(input_fn=test_input_fn)
  return [(sentence, prediction['labels'],prediction['logits_out']) for sentence, prediction in zip(in_sentences, predictions)]

In [79]:
pred_sentences = list(test['clean tweet'])
predictions = getPrediction2(pred_sentences,test_input_fn)
predictions[0]

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


('What does the new boss of Boeing need to do to turn the aerospace company around? Find out on our “Money Talks” podcast ',
 1,
 array([0.8782994 , 0.        , 0.        , 0.        , 1.0858985 ,
        0.10281204, 0.18187405, 0.        , 0.        , 0.74847645,
        0.        , 0.        , 2.2387722 , 0.        , 0.        ,
        0.        , 0.        , 1.0527737 , 0.        , 1.0788652 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.71054107, 2.1948037 ,
        0.        , 0.69700956, 0.        , 1.6327842 , 0.        ,
        0.1783132 , 2.0404184 , 0.        , 2.1926074 , 1.49878   ,
        0.7782261 , 0.        , 0.        , 0.        , 1.5330434 ,
        2.2594597 , 0.        , 0.24721095, 2.4739015 , 0.7997    ,
        1.031232  , 1.979018  , 0.5752012 , 0.        , 1.9438971 ,
        1.2093602 , 1.8312657 , 0.        , 0.        , 0.        ,
        1.876231  , 0.2949986 , 1.4892557 , 0.        ]

In [80]:
enc_labels = []
texts = []
for i in range(len(predictions)):
  enc_labels.append(label_list[predictions[i][1]]) # [0, 1] 
  texts.append(predictions[i][0].replace('\n', ' '))

import numpy as np
table = pd.DataFrame({'Text':texts, 'Label':enc_labels, 'True':test['target']})

In [81]:
test['target'].value_counts()

 0.0    193
-1.0    191
 1.0    184
Name: target, dtype: int64

In [82]:
table

,Text,Label,True
1371,What does the new boss of Boeing need to do to...,0,0.0
3024,Stocks making the biggest moves after hours: I...,1,1.0
7678,"Boeing plans to cut 787 Dreamliner output, job...",-1,-1.0
9766,"A surge in Zoom-bombing, or uninvited guests d...",-1,-1.0
433,presentation sued by california teachers over ...,-1,-1.0
9945,IBM's new CEO Arvind Krishna said he will focu...,1,0.0
5525,"""vanish, o night! at dawn, i will win!"" china'...",0,1.0
9962,Stocks making the biggest moves midday: JCPenn...,0,0.0
3657,IBM soars on strong earnings report.,1,1.0
7863,"Tesla will slash employee pay, and will furlou...",-1,-1.0


In [83]:
table['True'].value_counts()

 0.0    193
-1.0    191
 1.0    184
Name: True, dtype: int64

In [84]:
table['Label'].value_counts()

 1    204
-1    197
 0    167
Name: Label, dtype: int64

In [85]:
from sklearn.metrics import accuracy_score

accuracy_score(table['True'], table['Label'])

0.8450704225352113

# Reference:
Most of the code has been taken from the following resource:

* https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb
* https://analyticsindiamag.com/step-by-step-guide-to-implement-multi-class-classification-with-bert-tensorflow/
* https://github.com/AmalVijayan/BERT-For-Multi-Class-Classification
